In [3]:
from supporters import *
from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import SimpleITK as sitk
import imageio
import nibabel as nib
import cv2
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model('../../model/training/models/model_multi_v1.h5')

In [8]:
# Load sample and see what output is
sample = sitk.ReadImage('../../data/MM_WHS/train_images/ct_train_1006_image.nii.gz', sitk.sitkFloat32)
sample = sitk.GetArrayFromImage(sample)
sample.shape

(248, 512, 512)

In [9]:
sample = np.expand_dims(sample, axis=3)
sample = normalize(sample, axis=1)
sample.shape

(248, 512, 512, 1)

In [ ]:
y_pred = model.predict(sample)
y_pred

In [11]:
y_pred_argmax = np.argmax(y_pred, axis=3)

In [13]:
y_res2 = np.array([ 205., 420., 500., 550., 600., 820., 850.])
y_mapped = np.take(y_res2, y_pred_argmax)

In [20]:
np.unique(y_mapped)

array([205., 420., 500., 550., 850.])

In [21]:
import nibabel as nib
import numpy as np
from stl import mesh
from skimage import measure
from vedo import load, Volume, show, Plotter
from vedo.applications import RayCastPlotter
import os
import SimpleITK as sitk

In [ ]:
def segmented_reconstruction(y_mapped):
    whole_heart_flattened = y_mapped.flatten()
    unique_values, counts = np.unique(whole_heart_flattened, return_counts=True)
    label_arrays = []

    # Loop through the unique values
    for value in unique_values: 
        label_array = np.copy(y_mapped) 
        label_array[np.where(label_array != value)] = 0 
        label_arrays.append(label_array)

    # Loop through classes and create mesh
    for i, label_array in enumerate(label_arrays): 
        
        verts, faces, normals, values = measure.marching_cubes(label_arrays[i], 0)
        obj_3d = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
        
        for j, f in enumerate(faces):
            obj_3d.vectors[j] = verts[f]

        # Save the STL file with the name and the path
        obj_3d.save(f'../../3D_reconstruction/predicted_res/cardiac_class_{i+1}.stl')
        
segmented_reconstruction(y_mapped)